### 3.3.1 파인콘 API 초기화

In [19]:
!pip install pinecone-client==2.2.4 -q

In [1]:
from pinecone import Pinecone

# Pinecone API 키 설정
pinecone_api_key = "your-api-key-here"
pc = Pinecone(api_key=pinecone_api_key)

/Users/paesir/Desktop/llama-index/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 3.3.2 벡터 데이터 추가

In [2]:
from pinecone import ServerlessSpec
index_lists = [item.get("name") for item in pc.list_indexes().get("indexes")]

index_name = "my-index"

if index_name not in index_lists:
    pc.create_index(
        name=index_name,
        dimension=3, # 벡터의 차원 수
        metric="cosine", # 유사도 측정 방식 (cosine, euclidean, dotproduct 중 선택)
        spec=ServerlessSpec(
            cloud="aws", # 사용할 클라우드: aws 또는 gcp
            region="us-east-1" # 리전 설정 (free tier에서 지원되는 리전을 사용해야 함)
        )
    )

In [3]:
index = pc.Index(index_name)

# 벡터 데이터 삽입
vectors = [
    [0.1, 0.2, 0.3], # 첫 번째 데이터의 벡터
    [0.4, 0.5, 0.6], # 두 번째 데이터의 벡터
    [0.7, 0.8, 0.9], # 세 번째 데이터의 벡터
]

# 벡터를 인덱스에 추가
ids = ["doc1", "doc2", "doc3"]
index.upsert([(id, vector) for id, vector in zip(ids, vectors)])
print("아이템 임베딩을 인덱스에 추가했습니다.")

아이템 임베딩을 인덱스에 추가했습니다.


### 3.3.3 벡터 검색

In [4]:
import json
import time

# 벡터 추가 후 잠시 대기
time.sleep(3)  # 3초 대기 -> pinecone 인덱스 생성 시간 때문에 추가

# 인덱스 통계 확인
stats = index.describe_index_stats()
print(f"인덱스 내 벡터 개수: {stats['total_vector_count']}")

# 유사 벡터 검색
results = index.query(
    vector=[[0.1, 0.2, 0.3]], # 검색할 쿼리 벡터
    top_k=2, # 가장 유사한 2개의 벡터 반환
    include_metadata=False
)

formatted_results = {
    "검색된 문서 ID": results["matches"][0]["id"],
    "유사도 거리": results["matches"][0]["score"]
}
print("\n유사한 벡터 검색 결과:")
print(json.dumps(formatted_results, indent=4, ensure_ascii=False))

인덱스 내 벡터 개수: 3

유사한 벡터 검색 결과:
{
    "검색된 문서 ID": "doc1",
    "유사도 거리": 0.998880625
}


In [5]:
# 메타데이터와 함께 벡터 삽입
vectors = [
    ([0.1, 0.2, 0.3], {"category": "A", "year": 2020}),
    ([0.4, 0.5, 0.6], {"category": "B", "year": 2021}),
    ([0.7, 0.8, 0.9], {"category": "A", "year": 2022}),
]

ids = ["doc1", "doc2", "doc3"]
index.upsert([(id, vector, metadata) for id, (vector, metadata) in zip(ids, vectors)])

{'upserted_count': 3}

In [6]:
# 검색 벡터
query_vector = [0.1, 0.2, 0.25]

# 메타데이터 필터링 조건
filter_condition = {
    "category": {"$eq": "A"},
    "year": {"$gt": 2020}
}

# 검색
query_result = index.query(
    vector=query_vector,
    top_k=2,
    filter=filter_condition,
    include_metadata=True
)

In [7]:
formatted_results = {
    "검색된 문서 ID": query_result["matches"][0]["id"],
    "메타데이터": query_result["matches"][0]["metadata"]
}
print("\n유사한 벡터 검색 결과:")
print(json.dumps(formatted_results, indent=4, ensure_ascii=False))


유사한 벡터 검색 결과:
{
    "검색된 문서 ID": "doc3",
    "메타데이터": {
        "category": "A",
        "year": 2022.0
    }
}


### 3.3.5 임베딩 기반 라마인덱스 답변 생성

In [8]:
!pip install llama-index==0.11.11 \
    llama-index-vector-stores-pinecone==0.3.0 \
    llama-index-embeddings-huggingface==0.3.0 -q

In [9]:
from pinecone import Pinecone
from llama_index.core.schema import Document
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import Document
import os

In [10]:
# Pinecone API 키 설정
pc = Pinecone(api_key=pinecone_api_key)

index_name = "example"
spec = {
    "serverless": {
    "cloud": "aws",
    "region": "us-east-1"
    }
}
index_lists = [item.get("name") for item in pc.list_indexes().get("indexes")]
if index_name not in index_lists:
    pc.create_index(
        name=index_name,
        dimension=3,
        metric="cosine",
        spec=spec
    )
index = pc.Index(index_name)

# LlamaIndex에서 사용할 HuggingFace 임베딩 모델 설정
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

In [11]:
# 문서 데이터
documents = [
    "고양이는 작은 육식동물로, 주로 애완동물로 기릅니다. 민첩하고 장난기 있는 행동으로 유명합니다.",
    "강아지는 충성심이 강하고 친절한 동물로, 흔히 인간의 최고의 친구로 불립니다. 주로 애완동물로 기르고, 동반자로서 유명합니다.",
    "고양이와 강아지는 전 세계적으로 인기 있는 애완동물로, 각각 독특한 특징을 가지고 있습니다."
]
ids = ["doc1", "doc2", "doc3"]

# 문서를 Document 형식으로 변환
nodes = [Document(text=doc, id_=doc_id) for doc, doc_id in zip(documents, ids)]

# Pinecone 벡터 스토어 생성
vector_store = PineconeVectorStore(pinecone_index=index)

# LlamaIndex의 VectorStoreIndex 생성
index = VectorStoreIndex.from_documents(nodes, vector_store=vector_store, embed_model= embed_model)

In [12]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# API 키 설정
api_key = "your-api-key-here"  # 여기에 실제 API 키 입력
# 또는 환경 변수에서 가져오기
# api_key = os.environ.get("OPENAI_API_KEY")

# Settings에 LLM과 Embedding 모델 설정
Settings.llm = OpenAI(api_key=api_key, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(api_key=api_key)

query_engine = index.as_query_engine()

# 질의 수행
query_text = "고양이에 대해 알려줘"
response = query_engine.query(query_text)

# 결과 출력
print("\n[질의 결과]")
print(response)


[질의 결과]
고양이는 전 세계적으로 인기 있는 애완동물 중 하나로, 독특한 특징을 가지고 있습니다. 고양이는 일반적으로 독립적이고 호기심이 많은 성격을 지니며, 다양한 품종과 색상을 가지고 있습니다. 이들은 종종 사람들과의 유대감을 형성하지만, 강아지와는 다른 방식으로 애정 표현을 합니다. 고양이는 사냥 본능이 강해 장난감이나 움직이는 물체에 대한 관심이 높습니다.


### 3.3.6 라마인덱스 기반 답변 생성(임베딩 생략)

In [13]:
vector_store = PineconeVectorStore(pinecone_index=index)

# LlamaIndex의 VectorStoreIndex 생성
index = VectorStoreIndex.from_documents(nodes, vector_store=vector_store)

# 쿼리 엔진 생성
query_engine = index.as_query_engine()

# 질의 수행
query_text = "고양이에 대해 알려줘"
response = query_engine.query(query_text)

# 최종 응답 출력
print("[질의 결과]")
print(response)

# 응답 생성에 사용된 문서 확인
print("\n[응답에 사용된 문서]")
for i, node in enumerate(response.source_nodes, 1):
    print(f"{i}. {node.text}\n")

[질의 결과]
고양이는 작은 육식동물로 주로 애완동물로 기르며, 민첩하고 장난기 있는 행동으로 유명합니다.

[응답에 사용된 문서]
1. 고양이는 작은 육식동물로, 주로 애완동물로 기릅니다. 민첩하고 장난기 있는 행동으로 유명합니다.

2. 고양이와 강아지는 전 세계적으로 인기 있는 애완동물로, 각각 독특한 특징을 가지고 있습니다.

